In [1]:
# import required libriaries
from pathlib import Path
import json

from fpl_predictor.util import *

In [2]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./fpl_predictor/data')

In [3]:
# read in latest training set
df_train = pd.read_csv(path/'train_v8.csv', index_col=0, 
                       dtype={'season':str,
                              'comp':str,
                              'squad':str})

# path to current season
season_paths = [path/'2021-22']

# name for current season
season_names = ['2122']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('2122', 11, range(13,33))

In [4]:
# all_players#[all_players['full_name'] == 'Ben Davies']

In [5]:
# df_train = df_train.iloc[:-686]

In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == season_names[0]].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

28

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest.drop_duplicates(inplace=True)

In [9]:
# df_latest[df_latest['player'] == 'Chris Wood']
# lucas digne
# chris wood

In [10]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'team', 'play_proba']]

In [11]:
last_play_proba.head()

,player,team,play_proba
0,Kasper Schmeichel,Leicester City,1.0
1,Jonny Evans,Leicester City,0.0
2,Marc Albrighton,Leicester City,1.0
3,Nampalys Mendy,Leicester City,1.0
4,Danny Ward,Leicester City,1.0


In [12]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on=['player', 'team'], how='left')

In [13]:
df_latest.drop_duplicates(inplace=True)

In [14]:
df_train.drop_duplicates(inplace=True)

In [15]:
if last_gw==1:
    df_latest['relative_market_value_team_season'] = df_latest['relative_market_value_team']
    df_latest['relative_market_value_opponent_team_season'] = df_latest['relative_market_value_opponent_team']

In [16]:
# for when you're doing the season value properly
# relative_market_value_season = train_df[train_df['season'] == '2021'][['team', 'relative_market_value_team']].groupby(['team']).first()
# relative_market_value_season = relative_market_value_season.rename(columns={'relative_market_value_team': 'season_value'})
# relative_market_value_season

In [17]:
# now get futher stats from fbref
# standard stats
features = ['squad', 'comp', 'round', 'shots_total', 'shots_on_target', 'touches', 'pressures',
            'tackles', 'interceptions', 'blocks', 'xg', 'npxg', 'xa', 'sca', 
            'gca', 'passes_completed', 'passes', 'passes_pct',  'carries', 
            'dribbles_completed', 'dribbles']

In [18]:
# url for the premier league
url_base = 'https://fbref.com'
pl_history_page = '/en/comps/9/history/Premier-League-Seasons'
url = url_base + pl_history_page

# scrape the history table to get latest season
pl_history_table = get_table(url)
pl_history_urls = get_urls(['season'], 'season', pl_history_table, url_base)
season_urls = pl_history_urls[0:1]

df_enhanced = get_stats(features, season_urls, url_base)

2021-2022
https://fbref.com/en/comps/9/Premier-League-Stats
Manchester City
https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats
Ederson
Bernardo Silva
João Cancelo
Rúben Dias
Rodri
Aymeric Laporte
Backing off 0.2 seconds afters 1 tries calling function <function get_table at 0x000001D35145EA68> with args ('https://fbref.com/en/players/119b9a8e/matchlogs/2021-2022/summary/Aymeric-Laporte-Match-Logs',) and kwargs {}
Backing off 1.8 seconds afters 2 tries calling function <function get_table at 0x000001D35145EA68> with args ('https://fbref.com/en/players/119b9a8e/matchlogs/2021-2022/summary/Aymeric-Laporte-Match-Logs',) and kwargs {}
Backing off 2.6 seconds afters 3 tries calling function <function get_table at 0x000001D35145EA68> with args ('https://fbref.com/en/players/119b9a8e/matchlogs/2021-2022/summary/Aymeric-Laporte-Match-Logs',) and kwargs {}
Backing off 5.5 seconds afters 4 tries calling function <function get_table at 0x000001D35145EA68> with args ('https://fbref.com/en/p

Youri Tielemans
Wilfred Ndidi
James Maddison
Harvey Barnes
Luke Thomas
Jamie Vardy
Daniel Amartey
Timothy Castagne
Ademola Lookman
Kiernan Dewsbury Hall
Boubakary Soumaré
Ricardo Pereira
Marc Albrighton
Jonny Evans
Patson Daka
Kelechi Iheanacho
Jannik Vestergaard
Ryan Bertrand
Hamza Choudhury
James Justin
Ayoze Pérez
Nampalys Mendy
Kasey McAteer
Brighton
https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats
Robert Sánchez
Marc Cucurella
Leandro Trossard
Joël Veltman
Neal Maupay
Jakub Moder
Lewis Dunk
Adam Lallana
Yves Bissouma
Pascal Groß
Shane Duffy
Adam Webster
Tariq Lamptey
Dan Burn
Alexis Mac Allister
Solly March
Danny Welbeck
Enock Mwepu
Steven Alzate
Aaron Connolly
Jason Steele
Jeremy Sarmiento
Taylor Richards
Evan Ferguson
Jürgen Locadia
Newcastle Utd
https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats
Allan Saint-Maximin
Joelinton
Joe Willock
Jamaal Lascelles
Jonjo Shelvey
Fabian Schär
Martin Dúbravka
Ryan Fraser
Matt Ritchie
Callum Wilson
Javier Manquillo

In [19]:
# get the player name matching dictionary
with open(path/'match_list.json') as f:
  match_list = json.load(f)

match_list

{'Fabinho': 'Fabio Henrique Tavares',
 'Alisson': 'Alisson Ramses Becker',
 'Joël Matip': 'Joel Matip',
 'Joe Gomez': 'Joseph Gomez',
 'Naby Keïta': 'Naby Keita',
 'Adrián': 'Adrián San Miguel del Castillo',
 'Thiago Alcántara': 'Thiago Alcántara do Nascimento',
 'Allan': 'Allan Marques Loureiro',
 'Richarlison': 'Richarlison de Andrade',
 'Gylfi Sigurðsson': 'Gylfi Sigurdsson',
 'Séamus Coleman': 'Seamus Coleman',
 'André Gomes': 'André Filipe Tavares Gomes',
 'Bernard': 'Bernard Anício Caldeira Duarte',
 'Çağlar Söyüncü': 'Çaglar Söyüncü',
 'Bruno Fernandes': 'Bruno Miguel Borges Fernandes',
 'Fred': 'Frederico Rodrigues de Paula Santos',
 'Nemanja Matić': 'Nemanja Matic',
 'Alex Telles': 'Alex Nicolao Telles',
 'Pierre Højbjerg': 'Pierre-Emile Højbjerg',
 'Son Heung-min': 'Heung-Min Son',
 'Lucas Moura': 'Lucas Rodrigues Moura da Silva',
 'Érik Lamela': 'Erik Lamela',
 'Dele Alli': 'Bamidele Alli',
 'Carlos Vinícius': 'Carlos Vinicius Alves Morais',
 'Ederson': 'Ederson Santana de M

In [20]:
team_list = {}
team_list['Brighton'] = 'Brighton and Hove Albion'
team_list['Leeds United'] = 'Leeds'
team_list['Manchester Utd'] = 'Manchester United'
team_list['Newcastle Utd'] = 'Newcastle United'
team_list['Norwich City'] = 'Norwich'
team_list['Sheffield Utd'] = 'Sheffield United'
team_list['Tottenham'] = 'Tottenham Hotspur'
team_list['West Brom'] = 'West Bromwich Albion'
team_list['West Ham'] = 'West Ham United'
team_list['Wolves'] = 'Wolverhampton Wanderers'

In [33]:
# df_enhanced = df_enhanced.loc[df_enhanced[df_enhanced['round'].str[-2:] != 'on'].index]
# df_enhanced = df_enhanced.loc[df_enhanced[df_enhanced['squad'] != 'CSKA Moscow'].index]

In [27]:
# replace non-matching names/teams and extract gw as integer
df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)
df_enhanced['squad'] = df_enhanced['squad'].replace(team_list)
df_enhanced['round'] = df_enhanced['round'].str[-2:].astype('int')

In [28]:
# remove duplicated (happens when players transferred within season)
df_enhanced = df_enhanced.drop_duplicates()

In [29]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7649,2021-2022,Jon Rowe,2022-02-09,Norwich,Premier League,24,0,0,5,7,...,0.0,0,0,1,1,100.0,1,0,0,Jonathan Rowe
7650,2021-2022,Jon Rowe,2022-02-19,Norwich,Premier League,26,0,0,2,0,...,0.0,0,0,1,2,50.0,3,0,0,Jonathan Rowe
7651,2021-2022,Jon Rowe,2022-03-05,Norwich,Premier League,28,0,0,17,6,...,0.0,2,1,9,13,69.2,13,0,0,Jonathan Rowe
7652,2021-2022,Jon Rowe,2022-03-10,Norwich,Premier League,30,0,0,8,0,...,0.0,0,0,7,7,100.0,8,0,0,Jonathan Rowe


In [30]:
# create date field in same format as enhanced data
df_latest['date'] = df_latest['kickoff_time'].str[:10]

In [31]:
# check if any names aren't matching
enhanced_players = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['alt_name'].unique().tolist()
standard_players = df_latest['player'].unique().tolist()

unmatched_players = [x for x in enhanced_players if x not in standard_players]
unmatched_players

[]

In [34]:
# check if any teams aren't matching
enhanced_teams = df_enhanced[df_enhanced['date'].isin(df_latest['date'].unique())]['squad'].unique().tolist()
standard_teams = df_latest['team'].unique().tolist()

unmatched_teams = [x for x in enhanced_teams if x not in standard_teams]
unmatched_teams

[]

In [35]:
# any additional corrections
# match_list['Kayky Chagas'] = 'Kayky da Silva Chagas'
# match_list['Josh Dasilva'] = 'Pelenda Joshua Dasilva'
# match_list['Bruno Guimarães'] = 'Bruno Guimarães Rodriguez Moura'
# match_list['Samir Santos'] = 'Samir Caetano de Souza Santos'
# match_list['Hwang Hee-chan'] = 'Hee-Chan Hwang'

# match_list['Sadio Man�'] = 'Sadio Mané'
# match_list['Jo�l Matip'] = 'Joel Matip'
# match_list['Thiago Alc�ntara'] = 'Thiago Alcántara do Nascimento'
# match_list['Naby Ke�ta'] = 'Naby Keita'
# match_list['Luis D�az'] = 'Luis Díaz'
# match_list['Jos� S�'] = 'José Malheiro de Sá'
# match_list['Romain Sa�ss'] = 'Romain Saïss'
# match_list['N�lson Semedo'] = 'Nélson Cabral Semedo'
# match_list['Ra�l Jim�nez'] = 'Raúl Jiménez'
# match_list['Jo�o Moutinho'] = 'João Filipe Iria Santos Moutinho'
# match_list['R�ben Neves'] = 'Rúben Diogo da Silva Neves'
# match_list['Rayan A�t Nouri'] = 'Rayan Ait Nouri'
# match_list['Fernando Mar�al'] = 'Fernando Marçal'
# match_list['Emiliano Mart�nez'] = 'Emiliano Martínez'
# match_list['Emi Buend�a'] = 'Emiliano Buendía Stati'
# match_list['Fabian Sch�r'] = 'Fabian Schär'
# match_list['Martin D�bravka'] = 'Martin Dubravka'
# match_list['Miguel Almir�n'] = 'Miguel Almirón'
# match_list['Bruno Guimar�es'] = 'Bruno Guimarães Rodriguez Moura'
# match_list['Kiko Femen�a'] = 'Francisco Femenía Far'

In [36]:
# replace additional non-matching names
# df_enhanced['alt_name'] = df_enhanced['player'].replace(match_list)

In [37]:
# make sure there are no non matches after update
# enhanced_players =  df_enhanced[df_enhanced['round'] == last_gw]['alt_name'].unique().tolist()
# standard_players = df_latest['player'].unique().tolist()

# unmatched_players = [x for x in enhanced_players if x not in standard_players]
# unmatched_players

In [38]:
# if match list was updated then update the file
# json_file = json.dumps(match_list)
# f = open(path/"match_list.json","w")
# f.write(json_file)
# f.close()

In [39]:
# check there are no duplicates in the join fields for the enhanced data
assert(len(df_enhanced[df_enhanced.duplicated(subset=['alt_name', 'date'], keep=False)]) == 0)

In [40]:
df_latest[df_latest.duplicated(subset=['date','player','team'], keep=False)]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba,date


In [41]:
# check there are no duplicates in the join fields for the standard data
assert(len(df_train[df_train.duplicated(subset=['date','player','team'])]) == 0)

In [42]:
# df_train[df_train.duplicated(subset=['date','player','team'])]

In [43]:
# join standard and enhanced dataframes
df_latest_further = df_latest.merge(df_enhanced.drop(['player', 'season', 'round'], axis=1), 
                                   left_on=['date', 'player', 'team'], 
                                   right_on=['date', 'alt_name', 'squad'], how='left')

In [44]:
# drop the alt name field and check the length of the standard
# dataframe has not increased
df_latest_further = df_latest_further.drop(['alt_name'], axis=1)
assert(len(df_latest_further) == len(df_latest_further))

In [45]:
# add crowds field
df_latest_further['crowds'] = 1

In [46]:
assert(sum(df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()) == 0)

In [47]:
df_latest_further[df_latest_further['minutes'] > 0][df_latest_further[df_latest_further['minutes'] > 0]['xg'].isnull()]

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds


In [48]:
df_enhanced

,season,player,date,squad,comp,round,shots_total,shots_on_target,touches,pressures,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,alt_name
0,2021-2022,Ederson,2021-08-15,Manchester City,Premier League,1,0,0,22,0,...,0.0,2,0,20,22,90.9,10,0,0,Ederson Santana de Moraes
1,2021-2022,Ederson,2021-08-21,Manchester City,Premier League,2,0,0,14,0,...,0.0,0,0,12,12,100.0,6,0,0,Ederson Santana de Moraes
2,2021-2022,Ederson,2021-08-28,Manchester City,Premier League,3,0,0,37,0,...,0.0,0,0,31,37,83.8,25,0,0,Ederson Santana de Moraes
3,2021-2022,Ederson,2021-09-11,Manchester City,Premier League,4,0,0,26,0,...,0.0,0,0,22,24,91.7,10,0,0,Ederson Santana de Moraes
4,2021-2022,Ederson,2021-09-18,Manchester City,Premier League,5,0,0,40,0,...,0.0,0,0,39,39,100.0,28,0,0,Ederson Santana de Moraes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7649,2021-2022,Jon Rowe,2022-02-09,Norwich,Premier League,24,0,0,5,7,...,0.0,0,0,1,1,100.0,1,0,0,Jonathan Rowe
7650,2021-2022,Jon Rowe,2022-02-19,Norwich,Premier League,26,0,0,2,0,...,0.0,0,0,1,2,50.0,3,0,0,Jonathan Rowe
7651,2021-2022,Jon Rowe,2022-03-05,Norwich,Premier League,28,0,0,17,6,...,0.0,2,1,9,13,69.2,13,0,0,Jonathan Rowe
7652,2021-2022,Jon Rowe,2022-03-10,Norwich,Premier League,30,0,0,8,0,...,0.0,0,0,7,7,100.0,8,0,0,Jonathan Rowe


In [49]:
df_train_new = pd.concat([df_train, df_latest_further], ignore_index=True, axis=0, sort=False)

In [50]:
df_train_new[df_train_new['player'] == 'Harry Kane'].tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
129202,Harry Kane,25,4,90,Tottenham Hotspur,Wolverhampton Wanderers,NaN,NaN,True,2,...,0.1,2.0,0.0,16.0,26.0,61.5,33.0,2.0,4.0,1
129993,Harry Kane,26,4,90,Tottenham Hotspur,Manchester City,NaN,NaN,False,13,...,0.1,3.0,1.0,20.0,30.0,66.7,24.0,0.0,0.0,1
129994,Harry Kane,26,4,90,Tottenham Hotspur,Burnley,NaN,NaN,False,2,...,0.1,4.0,0.0,22.0,36.0,61.1,36.0,3.0,3.0,1
130880,Harry Kane,27,4,90,Tottenham Hotspur,Leeds,NaN,NaN,False,11,...,0.8,5.0,1.0,25.0,34.0,73.5,34.0,3.0,5.0,1
131632,Harry Kane,28,4,90,Tottenham Hotspur,Everton,NaN,NaN,True,13,...,0.3,2,1,26,32,81.3,31,2,3,1


In [51]:
df_train_new.tail(10)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,xa,sca,gca,passes_completed,passes,passes_pct,carries,dribbles_completed,dribbles,crowds
132295,Yerson Mosquera Valdelamar,28,2,0,Wolverhampton Wanderers,Crystal Palace,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
132296,Yerson Mosquera Valdelamar,28,2,0,Wolverhampton Wanderers,Watford,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
132297,Yoane Wissa,28,3,8,Brentford,Norwich,NaN,NaN,False,1,...,0.0,0,0,5,5,100.0,2,0,0,1
132298,Youri Tielemans,28,3,90,Leicester City,Leeds,NaN,NaN,True,3,...,0.0,0,0,27,32,84.4,30,2,5,1
132299,Yves Bissouma,28,3,0,Brighton and Hove Albion,Newcastle United,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
132300,Zach Awe,28,2,0,Arsenal,Watford,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
132301,Zack Steffen,28,1,0,Manchester City,Manchester United,NaN,NaN,True,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
132302,Zak Swanson,28,2,0,Arsenal,Watford,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
132303,Álvaro Fernández,28,1,0,Brentford,Norwich,NaN,NaN,False,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
132304,Çaglar Söyüncü,28,2,90,Leicester City,Leeds,NaN,NaN,True,8,...,0.0,0,0,51,60,85.0,39,0,0,1


In [52]:
df_latest_further.shape

(1009, 57)

In [53]:
df_train_new.shape

(132305, 59)

In [54]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v8.csv')

In [55]:
# get fixtures
fixtures = pd.read_csv(path/'2021-22/fixtures.csv')
team_dict = teams[['team_2122', 'team']].dropna().set_index('team_2122').to_dict()['team']

fixtures['home_team'] = fixtures['team_h'].replace(team_dict)
fixtures['away_team'] = fixtures['team_a'].replace(team_dict)
fixtures = fixtures[['home_team', 'away_team', 'kickoff_time', 'event']]
fixtures = fixtures.rename(columns={'event':'gw'})


# use these two lines if updating play_proba later in week
# current_gw = last_gw
# df_train_new = pd.read_csv(path/'train_v7.csv', index_col=0, dtype={'season':str,
#                                                                     'comp':str,
#                                                                     'squad':str})

# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = fixtures[fixtures['gw'] >= current_gw].sort_values(by=['gw'])
fixtures['match_no'] = np.arange(fixtures.shape[0])

In [56]:
fixtures.head(50)

,home_team,away_team,kickoff_time,gw,match_no
284,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,29.0,0
296,Everton,Newcastle United,2022-03-17T19:45:00Z,29.0,1
295,Arsenal,Liverpool,2022-03-16T20:15:00Z,29.0,2
293,Crystal Palace,Manchester City,2022-03-14T20:00:00Z,29.0,3
292,Arsenal,Leicester City,2022-03-13T16:30:00Z,29.0,4
291,West Ham United,Aston Villa,2022-03-13T14:00:00Z,29.0,5
294,Brighton and Hove Albion,Tottenham Hotspur,2022-03-16T19:30:00Z,29.0,6
289,Leeds,Norwich,2022-03-13T14:00:00Z,29.0,7
288,Everton,Wolverhampton Wanderers,2022-03-13T14:00:00Z,29.0,8
287,Chelsea,Newcastle United,2022-03-13T14:00:00Z,29.0,9


In [57]:
# add universal team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [58]:
fixtures.head()

,gw,match_no,home_team,away_team,kickoff_time,home_team_code,away_team_code
0,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,36,14
1,29.0,1,Everton,Newcastle United,2022-03-17T19:45:00Z,11,4
2,29.0,2,Arsenal,Liverpool,2022-03-16T20:15:00Z,3,14
3,29.0,3,Crystal Palace,Manchester City,2022-03-14T20:00:00Z,31,43
4,29.0,4,Arsenal,Leicester City,2022-03-13T16:30:00Z,3,13


In [59]:
# all_players.replace(to_replace='None', value=1, inplace=True)

In [60]:
all_players[all_players['play_proba_2122'] == '0']

,full_name,play_proba_2122,position_2122,id_2122,cost_2122,team_2122,web_name_2122
1,Rúnar Alex Rúnarsson,0,1,2,40,3,Rúnarsson
2,Willian Borges Da Silva,0,3,3,63,3,Willian
3,Pierre-Emerick Aubameyang,0,4,4,96,3,Aubameyang
7,Pablo Marí,0,2,8,42,3,Marí
8,Héctor Bellerín,0,2,9,48,3,Bellerín
...,...,...,...,...,...,...,...
704,Christian Marques,0,2,549,40,39,Marques
706,Louie Moulden,0,1,611,40,39,Moulden
709,Bruno André Cavaco Jordão,0,3,661,45,39,Jordão
711,Dion Sanderson,0,2,686,40,39,Sanderson


In [61]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'match_no', 'home_team', 'away_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [62]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_2122', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'match_no', 'away_team', 'home_team', 'kickoff_time', 
                   'full_name', 'position_2122', 'cost_2122', 'play_proba_2122', 
                   'web_name_2122']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_2122':'position',
                        'cost_2122':'price',
                        'play_proba_2122':'play_proba',
                        'web_name_2122':'short_name'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [63]:
# look at away players
away_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,29.0,0,Liverpool,Brighton and Hove Albion,2022-03-12T12:30:00Z,James Milner,3,49,100,Milner,False
1,29.0,0,Liverpool,Brighton and Hove Albion,2022-03-12T12:30:00Z,Jordan Henderson,3,50,100,Henderson,False
2,29.0,0,Liverpool,Brighton and Hove Albion,2022-03-12T12:30:00Z,Adrián San Miguel del Castillo,1,43,100,Adrián,False
3,29.0,0,Liverpool,Brighton and Hove Albion,2022-03-12T12:30:00Z,Joel Matip,2,49,100,Matip,False
4,29.0,0,Liverpool,Brighton and Hove Albion,2022-03-12T12:30:00Z,Thiago Alcántara do Nascimento,3,53,100,Thiago,False


In [64]:
# look at home players
home_df.head()

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home
0,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Adam Lallana,3,53,100,Lallana,True
1,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Jason Steele,1,40,100,Steele,True
2,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Pascal Groß,3,57,100,Groß,True
3,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Shane Duffy,2,41,100,Duffy,True
4,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Davy Pröpper,3,45,0,Pröpper,True


In [65]:
# # calculate average minutes over last 5 gameweeks
# # first get last 5 gameweeks in current season
# recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
#                            (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# # recent_mins = df_train_new[((df_train_new['gw'] >= 40 - 5) & 
# #                            (df_train_new['season'] == '1920')) | 
# #                            ((df_train_new['gw'] >= 1) & 
# #                            (df_train_new['season'] == '2021'))][['player', 'minutes', 'play_proba']]

# # exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
# recent_mins = recent_mins[recent_mins['play_proba'] == 1]

# len(recent_mins)

# # average for each player
# recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

# recent_mins

In [66]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '2122'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = season_names[0]

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# # add minutes based on last five fixtures
# remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
# remaining_season_df['minutes'].fillna(0, inplace=True)

# # multiply minutes by play probability
# remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']

In [67]:
remaining_season_df.head(50)

,gw,match_no,team,opponent_team,kickoff_time,player,position,price,play_proba,short_name,was_home,relative_market_value_team,relative_market_value_opponent_team,season
0,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Adam Lallana,3,5.3,1.0,Lallana,True,NaN,NaN,2122
1,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Jason Steele,1,4.0,1.0,Steele,True,NaN,NaN,2122
2,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Pascal Groß,3,5.7,1.0,Groß,True,NaN,NaN,2122
3,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Shane Duffy,2,4.1,1.0,Duffy,True,NaN,NaN,2122
4,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Davy Pröpper,3,4.5,0.0,Pröpper,True,NaN,NaN,2122
5,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Lewis Dunk,2,4.8,1.0,Dunk,True,NaN,NaN,2122
6,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Solomon March,3,5.3,1.0,March,True,NaN,NaN,2122
7,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Adam Webster,2,4.3,0.0,Webster,True,NaN,NaN,2122
8,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Joël Veltman,2,4.2,1.0,Veltman,True,NaN,NaN,2122
9,29.0,0,Brighton and Hove Albion,Liverpool,2022-03-12T12:30:00Z,Neal Maupay,4,6.5,1.0,Maupay,True,NaN,NaN,2122


In [68]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')